# Snowpocalypse '19
February 10, 2019  
David Shean

## Objectives
* Explore spatial relationships in time series of field observations
* Learn about dynamic data fetching, ingestion into Pandas/GeoPandas and basic spatial analysis
* Explore some simple interpolation routines
* Explore some fundamental concepts and metrics for snow science

## Read a bit about SNOTEL data for the Western U.S.

https://www.wcc.nrcs.usda.gov/snow/

This is actually a nice web interface, with some advanced querying and interactive visualization.  You can also download formatted ASCII files for later analysis.  This is great for one-time projects, but it's nice to have deterministic code that can be updated as new data appear, without manual steps.

### About SNOTEL sites:
* https://www.wcc.nrcs.usda.gov/about/mon_automate.html
* https://www.wcc.nrcs.usda.gov/snotel/snotel_sensors.html

## CUAHSI WOF server and `ulmo` for Python queries

### Acronym soup
* SNOTEL = Snow Telemetry
* CUAHSI = Consortium of Universities for the Advancement of Hydrologic Science, Inc
* WOF = WaterOneFlow
* WSDL = Web Services Description Language
* USDA = United States Department of Agriculture
* NRCS = National Resources Conservation Service
* AWDB = Air-Water Database

We are going to use a server set up by CUAHSI to serve the SNOTEL data, using a standardized database storage format and query structure.  You don't need to worry about this, but please quickly review the following:
* http://hiscentral.cuahsi.org/pub_network.aspx?n=241 
* http://his.cuahsi.org/wofws.html

### Python options

There are a few packages out there that offer convenience functions to query the online SNOTEL databases and unpack the results.  You can also write your own queries using the Python `requests` module and some built-in XML parsing libraries
* climata (https://pypi.org/project/climata/) - last commit Sept 2017 (not a good sign)
* ulmo (https://github.com/ulmo-dev/ulmo) - last commit a Feb 2019 (but not well maintained, and will be superseded soon)

Hopefully not overwhelming amount of information - let's just go with ulmo for now.  I've done most of the work to prepare functions for querying and processing the data.  Once you wrap your head around all of the acronyms, it's pretty simple, basically running a few functions here: https://ulmo.readthedocs.io/en/latest/api.html#ulmo-cuahsi-wof

I did discover a bug on the CUAHSI server side with the hourly SNOTEL data.  This works fine through climata and the https://wcc.sc.egov.usda.gov/awdbWebService/services?WSDL source), but I don't have time to redo everything with climata.  

So we're just going to use ulmo with daily data for this exercise, but please feel free to experiment with my code, with climata, or parsing files you download via the web interface.

Note that support for ulmo is ending, and will be superseded by another package called Quest (https://github.com/ulmo-dev/ulmo/issues/161)

### Important ulmo installation note

The current conda-forge build of ulmo requires an older version of pandas.  Also, some commits in the past few weeks fixed some bugs.  I've been told there will be a new conda-forge build next week, but we have geospatial data that needs analysis now!  So we're going to use a development version of ulmo, straight from the github source.  This is a good exercise, and will show you how to grab source code directly from github for local use and development.

To install:
1. Open a terminal on the Jupyterhub and `git clone https://github.com/ulmo-dev/ulmo.git` somewhere in your home directory
2. `cd ulmo`
3. `pip install -e .` (this will run a "developer" install, which means you can modify the source code and use updated versions in real-time)
4. Restart the kernel in your notebook and you should be all set to `import ulmo`
5. Note that if your Jupyterlab server restarts, you may need to repeat #3

In [ ]:
#%matplotlib widget
%matplotlib inline
import os
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import ulmo
plt.rcParams['figure.figsize'] = [10, 8]

## CUAHSI WOF server information

In [ ]:
#http://his.cuahsi.org/wofws.html
wsdlurl = 'http://hydroportal.cuahsi.org/Snotel/cuahsi_1_1.asmx?WSDL'
#wsdlurl = "http://worldwater.byu.edu/interactive/snotel/services/index.php/cuahsi_1_1.asmx?WSDL"  # WOF 1.1

## Get information about all of the SNOTEL sites from the server

In [ ]:
sites = ulmo.cuahsi.wof.get_sites(wsdlurl)

## Take a moment to inspect the output

* What is the `type`?
* What happens when you print?

## Store as a Pandas DataFrame
* See the `from_dict` function
* Use `orient` option so the sites comprise the DataFrame index, with columns for 'name', 'elevation_m', etc

## Clean up the DataFrame and prepare Point geometry objects
* Convert 'location' column (contains dictionary with 'latitude' and 'longitude' values) to Shapely Points
* Store as a new 'geometry' column
* Drop the 'location' column, as this is no longer needed
* Update the dtype of the 'elevation_m' column to float

## Create a GeoDataFrame with appropriate crs definition

* Take a moment to familiarize yourself with the DataFrame structure and different fields.
* Note that the index is a set of strings with format 'SNOTEL:1000_OR_SNTL'
* Note the number of records

## Create a scatterplot showing elevation values of all sites

## Remove any records with incorrect coordinates of (0,0)
* Note the updated number of records

## Remove the Alaska (AK) points to isolate points over Western U.S.
* Can use a spatial filter (see GeoPandas cx indexer functionality) or remove points where the site name contains 'AK'
* Note the number of records

## Update your scatterplot as sanity check

## Create an interactive plot with basemap using folium

We haven't discussed as a class yet, but this is a simple, effective interactive visualization package (alternative to matplotlib)

See the example here: https://python-visualization.github.io/folium/docs-v0.6.0/quickstart.html

For your plot:
* Create a map centered on the centroid of your filtered SNOTEL sites (remember GeoPandas `unary_union`)
* Use the 'Stamen Terrain' basemap layer, experiment with `zoom_start` level
* Export your GeoDataframe using `to_json()`, then load using `folium.features.GeoJson`
* Add the points to the map

## Extra Credit (if you have time after finishing the lab, and/or are interested in exploring folium)

* Create an interactive `folium` map with a MarkerCluster that displays the SNOTEL site name and/or ID
* This example is likely useful: https://ocefpaf.github.io/python4oceanographers/blog/2015/12/14/geopandas_folium/

## Create a histogram of SNOTEL site elevations
* What is the highest SNOTEL site in the Western U.S.?  How about WA?
* Do these elevations seem to provide a good sample of elevations where snow accumulates? (just think about this for a minute, I'm hoping we can revisit when we start working with rasters and DEM data)

## Reproject the sites GeoDataFrame to an equal-area projection
* Can use the same projection from the GLAS example, or recompute based on bounds and center of SNOTEL sites

## Create a scatterplot and overlay the state polygons

## Isolate WA sites and retrieve Snow Depth data
* Could do a spatial join with WA polygon, as we did in the GLAS example
* But probably easiest to filter records with 'WA' in the index
    * Note: need to convert to `str`, then use `contains`
* Sanity check - note number of records and create a quick scatterplot to verify

## Create a histogram plot showing elevation of all SNOTEL sites in Western US and the WA sample
* What do you notice about the WA sample?

## Get the highest site in WA and assign to a variable named `sitecode`

## Query the server for information about this site

In [ ]:
site_info = ulmo.cuahsi.wof.get_site_info(wsdlurl, sitecode)

## Inspect the returned information
* Note available data series 

In [ ]:
site_info['series'].keys()

## Let's consider the 'SNOTEL:SNWD_D' variable (Daily Snow Depth)
* Assign to a variable named `variablecode`
* Get some information about the variable
* Note the units, nodata value, etc.

In [ ]:
#variablecode = 'SNOTEL:WTEQ_H'
#variablecode = 'SNOTEL:WTEQ_D'
#variablecode = 'SNOTEL:SNWD_H'
variablecode = 'SNOTEL:SNWD_D'
ulmo.cuahsi.wof.get_variable_info(wsdlurl, variablecode)

## Define a function to fetch data!

In [ ]:
today = datetime.today().strftime('%Y-%m-%d')

def fetch(sitecode, variablecode='SNOTEL:SNWD_D', start_date='1950-10-01', end_date=today):
    print(sitecode, variablecode, start_date, end_date)
    values_df = None
    try:
        #Request data from the server
        site_values = ulmo.cuahsi.wof.get_values(wsdlurl, sitecode, variablecode, start=start_date, end=end_date)
        #Convert to a Pandas DataFrame   
        values_df = pd.DataFrame.from_dict(site_values['values'])
        #Parse the datetime values to Pandas Timestamp objects
        values_df['datetime'] = pd.to_datetime(values_df['datetime'], utc=True)
        #Set the DataFrame index to the Timestamps
        values_df = values_df.set_index('datetime')
        #Convert values to float and replace -9999 nodata values with NaN
        values_df['value'] = pd.to_numeric(values_df['value']).replace(-9999, np.nan)
        #Remove any records flagged with lower quality
        values_df = values_df[values_df['quality_control_level_code'] == '1']
    except:
        print("Unable to fetch %s" % variablecode)

    return values_df

## Get the full 'SNOTEL:SNWD_D' record for the highest station in WA without specifying start and end dates
* Inspect the results
* What are the first and last dates returned?

In [ ]:
values_df = fetch(sitecode, variablecode)

## Create a quick plot to view the time series
* What do you notice?

# Get all daily snow depth records for all sites in WA
* Note that this could take some time to run (10-20 min, depending on server load)
* Loop through all sites names in the WA GeoDataFrame and fetch
* Store in a dictionary
* Convert the dictionary to a Pandas Dataframe

## Save the DataFrame, so you don't have to download again
* Can use a number of different formats, easiest to use a "pickle": https://www.pythoncentral.io/how-to-pickle-unpickle-tutorial/
* Define a unique filename for the dataset (e.g., `pkl_fn='snotel_wa_snwd_d.pkl'`)
* Write the DataFrame to disk
* Read it to a temporary variable and verify that everything looks good

In [ ]:
pkl_fn = 'snotel_wa_snwd_d.pkl'
#pkl_fn = variablecode.replace(':','_')+'_'+start_date+'_'+end_date+'.pkl'

if os.path.exists(pkl_fn):
    wa_dict = pd.read_pickle(pkl_fn)
else:
    #Define an empty dictionary to store returns for each site
    value_dict = {}
    for sitecode in sites_gdf_wa.index:
        out = fetch(sitecode, variablecode, start_date, end_date)
        if out is not None:
            value_dict[sitecode] = out['value']
    #Convert the dictionary to a DataFrame, automatically handles different datetime ranges (nice!)
    wa_dict = pd.DataFrame.from_dict(value_dict)
    #Write out
    wa_dict.to_pickle(pkl_fn)

## Inspect the DataFrame
* Note structure, number of timestamps, NaNs

## Convert snow depth inches to cm

## Compute daily snow accumulation at all stations
* See the `diff` function
* Make sure you specify the axis properly to difference over time (not station to station), and sanity check
* Create a plot of differences for all sites
    * Adjusting the ylim appropriately
    * Probably best to turn off the legend in your plot call

## How might you filter these differences to remove artifacts and outliers?
* If a single station shows an increase of 2 m, but all others show a decrease, is that realistic?
* Create a plot showing the median difference values across all stations for each day
* Do you think you can confidently identify large snow accumulation events using these difference values?
* Could likely design a filter to remove outliers using mean +/- (3 * std) for all stations
    * Maybe come back to this if you have time later, for now, just note the presence of measurement errors

## Get the count of valid records (not NaN) available for each station
* Create a bar plot, sorted from shortest to longest record
* What is the longest record available for SNWD_D?

## Add the count dataseries as a new column in the WA sites GeoDataframe
* Should be easy, let Pandas do the work to match site index values
* Verify your site DataFrame now has a count column

## Add a column for the long-term median snow depth at each site
* Note: to do this properly, probably want to remove any values near 0 before computing the median

## Add a column for the long-term max snow depth on record for each site
* Might need to be careful about measurement errors here - maybe look at values and filter obvious outliers

## Add a column for the daily difference (snow accumulation) at all sites for the recent Feb 8 snow event
* Note, you may need to use `mydataframe.loc[pd.Timestamp('YYYY-MM-DD')]`

In [ ]:
sites_gdf_wa['2019-02-05_accum'] = wa_dict.diff(axis=0).loc[pd.Timestamp('2019-02-05')]

## Add a column for the snow depth on the most recent day in the record
* Use the last index value (should be today, or whenever you ran the query)
* Note that the index is not a string, it is a Pandas Timestamp object: `pd.Timestamp('2019-02-06 00:00:00')`

In [ ]:
ts = wa_dict.index[-1]
print(ts)
print(type(ts))
#sites_gdf_wa[ts] = wa_dict.iloc[-1]
sites_gdf_wa['2019-02-06'] = wa_dict.iloc[-1]
sites_gdf_wa.head()

## Create some scatterplots to review these metrics
* Need to remove NaN records on the fly before plotting - see the `dropna()` method
    * Best to apply after isolating the column you want to plot, consider `mydataframe[['median', 'geometry]].dropna().plot(column='median', ...)`

## Compute and plot at least one additional metric of interest for snow depth at the WA sites
* Consider aggregation over time and/or space
* Do some brief analysis and interpretation

## Can you identify the SNOTEL site that received the most snow accumulation during the Feb 2019 storm(s)?

## Snow depth vs. elevation analysis
* Let's look at snow depth from most recent day in the record
* Create a quick scatterplot of elevation vs. snow depth for all sites on this day (or several days if desired)
* Do you see a relationship?  

## Interpolate recent accumulation pattern for the full WA state domain
* Note that you wouldn't want to do this in practice for such a sparse sample, but let's explore for purposes of illustration
* We'll use the `scipy.interpolate.griddata` function here, using 'nearest' to start
* Try a few different interpolation methods

In [ ]:
import numpy as np
import scipy.interpolate

In [ ]:
#Column name of variable to interpolate
var = '2019-02-08_accum'
#Extract the column and geometry, drop NaNs
sites_gdf_wa_dropna = sites_gdf_wa[[var,'geometry']].dropna()
#Pull out (x,y,val)
x = sites_gdf_wa_dropna.geometry.x
y = sites_gdf_wa_dropna.geometry.y
z = sites_gdf_wa_dropna[var]
#Get min and max values
zlim = (z.min(), z.max())

In [ ]:
#Compute the spatial extent of the points - we will interpolate across this domain
bounds = sites_gdf_wa_dropna.total_bounds
#Spatial interpolation step of 1 km
dx,dy = (1000,1000)
#Create 1D arrays of grid cell coordinates in the x and y directions
xi = np.arange(np.floor(bounds[0]), np.ceil(bounds[2]),dx)
yi = np.arange(np.floor(bounds[1]), np.ceil(bounds[3]),dy)

In [ ]:
#Create 2D grids from the xi and yi grid cell coordinates
xx, yy = np.meshgrid(xi, yi)
#Interpolate values
zi = scipy.interpolate.griddata((x,y), z, (xx, yy), method='nearest')

In [ ]:
#Create a plot
f, ax = plt.subplots()
#Define extent of the interpolated grid in projected coordinate system, using matplotlib extent format (left, right, bottom, top)
mpl_extent = [bounds[0], bounds[2], bounds[1], bounds[3]]
#Plot the interpolated grid, providing known extent
#Note: need the [::-1] to flip the grid in the y direction
ax.imshow(zi[::-1,], cmap='inferno', extent=mpl_extent, clim=zlim)
#Overlay the original point values with the same color ramp
sites_gdf_wa.plot(ax=ax, column=var, cmap='inferno', markersize=20, edgecolor='w', vmin=zlim[0], vmax=zlim[1], legend=True)
#Make sure aspect is equal
ax.set_aspect('equal')

# Extra credit (or, some additional ideas to explore)

If you have some time (or curiosity), feel free to explore some of these, or define your own questions.

1. Compute snow depth statistics across all sites grouping by Water Year
2. Long-term median snowdepth for each day of the year, how does 2019 compare (percent of long-term median)
4. Identify date of first major snow accumulation event each year, date of max snow depth, date of snow disappearance - any evolution over time?
5. Split sites into elevation bands and analyze various metrics
6. Explore other interpolation methods for sparse data
7. Create an animated map of daily accumulation in WA for the past two weeks
8. Look at other variables for the SNOTEL sites (WTEQ_D, SNWD_H)
    * Note that WTEQ_D time series begin much earlier than SNWD_D
9. Rerun for the full Western U.S.